Interacting Proteins

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as p

In [2]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


In [82]:
prot = gbm.get_proteomics()
p = list(prot.columns)
len(p)

11141

In [4]:
gene = 'PTEN'

In [83]:
mut_type_gbm = gbm.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


Deletion          81
Wildtype_Tumor    16
Name: Mutation, dtype: int64

In [84]:
cols = list(del_wt.columns[:-1])

g = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
g_pval = g.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
g_pval = g_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
g_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,Gbm_P_Value
0,ARMH3,5.396032e-11
1,CUTC,1.593480e-10
2,PIP4K2A,1.009419e-09
3,CUL2,1.122076e-09
4,GDI2,1.302273e-09
...,...,...
11136,ZNF805,NaN
11137,ZNF813,NaN
11138,ZNF814,NaN
11139,ZNF888,NaN


In [85]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

gbm_d = {}

for prot in p:
    dif_gbm = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    gbm_d[prot] = dif_gbm
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif_gbm)

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm_Median'])
gbm_df = gbm_df.reset_index().rename(columns={'index':'Proteomics'})
gbm_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,Gbm_Median
0,A1BG,0.245016
1,A2M,0.680437
2,AAAS,-0.128834
3,AACS,0.005207
4,AADAT,0.200270
...,...,...
11136,ZXDC,0.010868
11137,ZYG11B,-0.185994
11138,ZYX,-0.314468
11139,ZZEF1,0.131491


In [86]:
g_merged = g_pval.merge(gbm_df, on='Proteomics',how='inner')
g_merged

,Proteomics,Gbm_P_Value,Gbm_Median
0,ARMH3,5.396032e-11,0.405134
1,CUTC,1.593480e-10,0.553255
2,PIP4K2A,1.009419e-09,0.838882
3,CUL2,1.122076e-09,0.586396
4,GDI2,1.302273e-09,0.610188
...,...,...,...
11136,ZNF805,NaN,NaN
11137,ZNF813,NaN,NaN
11138,ZNF814,NaN,0.265936
11139,ZNF888,NaN,NaN


Endo

In [9]:
prot = endo.get_proteomics()
p = list(prot.columns)
len(p)

10999

In [10]:
#Endo ttest

mut_type = endo.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt['Mutation'].value_counts()
#trunc_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 7)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Truncation        53
Wildtype_Tumor    15
Name: Mutation, dtype: int64

In [11]:
cols = list(trunc_wt.columns[:-1])

e_pval = u.wrap_ttest(trunc_wt, 'Mutation', cols, return_all = True)
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})
e_pval = e_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
e_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,En_P_Value
0,NOL10,8.999599e-08
1,ABT1,3.141336e-07
2,UTP25,4.308134e-07
3,TOPBP1,4.885533e-07
4,RIF1,9.004982e-07
...,...,...
10994,ZNF586,NaN
10995,ZNF630,NaN
10996,ZNF772,NaN
10997,ZNF79,NaN


In [12]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - trunc_med[prot+'_proteomics']
    en_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', trunc_med[prot+'_proteomics'], '=',dif)

en_df = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_df = en_df.reset_index().rename(columns={'index':'Proteomics'})
en_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,En_Median
0,A1BG,0.06100
1,A2M,-0.07400
2,A2ML1,-0.22300
3,A4GALT,-0.03995
4,AAAS,0.19200
...,...,...
10994,ZXDC,0.42800
10995,ZYG11B,0.19420
10996,ZYX,-0.21600
10997,ZZEF1,-0.05320


In [13]:
e_merged = e_pval.merge(en_df, on='Proteomics',how='inner')
e_merged

,Proteomics,En_P_Value,En_Median
0,NOL10,8.999599e-08,0.44200
1,ABT1,3.141336e-07,0.48510
2,UTP25,4.308134e-07,0.73450
3,TOPBP1,4.885533e-07,0.61795
4,RIF1,9.004982e-07,0.54640
...,...,...,...
10994,ZNF586,NaN,NaN
10995,ZNF630,NaN,-0.99250
10996,ZNF772,NaN,NaN
10997,ZNF79,NaN,NaN


Ovarian

In [14]:
o = cptac.Ovarian()

In [15]:
gene = 'PTEN'

prot = o.get_proteomics()
prot = o.reduce_multiindex(prot, levels_to_drop = 1)
p = list(prot.columns)
len(p)
prot

cptac warning: Due to dropping the specified levels, dataframe now has 1162 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)


Name,A1BG,A2M,A2ML1,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,...,ZSCAN31,ZSCAN32,ZSWIM8,ZW10,ZWILCH,ZWINT,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01OV007,0.133634,0.765120,-0.778849,0.118236,0.112740,0.054942,0.313055,-0.786270,-0.384680,0.064017,...,-0.382350,-0.323516,0.074209,0.232999,-0.195793,NaN,-0.145053,0.590217,0.019414,NaN
01OV017,-0.432786,-0.461906,-0.533497,0.041800,0.604114,0.351294,0.546605,0.603935,-0.235374,0.059084,...,-0.008922,0.032004,0.016276,0.587911,0.574339,NaN,0.075401,-0.437564,-0.208831,NaN
01OV018,-0.013943,0.065842,NaN,-0.010514,0.541935,0.169826,-0.023032,-0.057549,-0.397627,0.199581,...,0.003784,0.002237,0.206993,0.111037,1.020321,NaN,-0.245864,0.008411,0.130504,-0.367433
01OV023,1.234203,1.512082,-0.703050,0.181152,0.087719,0.226861,-0.100766,-0.516022,-0.456512,-0.257629,...,-0.277046,NaN,-0.529948,0.034713,0.149523,-0.313370,-0.507473,0.221818,0.278313,-1.008998
01OV026,-0.482872,0.343872,-1.150936,0.315155,0.097235,-0.160208,-0.125165,-0.768776,-0.372368,0.450086,...,-0.096284,-0.752819,-0.181027,0.179612,0.007854,-0.106049,0.377063,-0.295592,0.011997,-0.236003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17OV001.N,-0.012139,0.437837,-0.704253,0.411204,-0.418927,0.016434,0.504605,0.022115,-0.163317,0.290281,...,-0.351444,0.812955,0.010818,0.321095,-0.612827,-0.149425,0.017523,0.442398,0.088551,-0.614771
17OV002.N,1.949941,1.803647,0.924203,-0.025210,0.593528,-0.427610,-0.094656,0.082444,0.065310,-0.322335,...,0.483503,NaN,-0.530407,-0.157267,0.716779,-0.676827,-0.569823,0.640503,0.763645,-1.370872
17OV003.N,1.523833,3.418092,0.281521,-0.148852,-0.051749,0.297993,0.160205,-0.318083,-0.382827,0.420826,...,-0.070859,NaN,0.778621,0.425679,-0.048390,NaN,0.675466,0.267934,0.583772,0.501873


In [16]:
mut_type = o.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = o.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = o.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
len(prot_and_mutations.columns)

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: Due to dropping the specified levels, dataframe now has 1162 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packa

10081

In [17]:
cols = list(del_wt.columns[:-1])

o = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
o = o.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Ov_P_Value'})
o_pval = o.replace(to_replace = r'_proteomics$', value = '', regex = True)
o_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,Ov_P_Value
0,MMS19,1.058914e-08
1,RACK1,8.573522e-05
2,PI4K2A,9.310383e-05
3,WAPL,1.724680e-04
4,IDE,2.011475e-04
...,...,...
10072,SAXO2,NaN
10073,SH3GL3,NaN
10074,SLC22A3,NaN
10075,SSX2B,NaN


In [18]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

ov_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    ov_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

o_df = pd.DataFrame.from_dict(ov_d, orient='index', columns=['Ov_Median'])
o_df = o_df.reset_index().rename(columns={'index':'Proteomics'})
#o_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [19]:
o_merged = o_pval.merge(o_df, on='Proteomics',how='inner')
o_merged

,Proteomics,Ov_P_Value,Ov_Median
0,MMS19,1.058914e-08,0.324897
1,RACK1,8.573522e-05,0.191250
2,PI4K2A,9.310383e-05,0.298399
3,WAPL,1.724680e-04,0.218775
4,IDE,2.011475e-04,0.208025
...,...,...,...
10072,SAXO2,NaN,0.099993
10073,SH3GL3,NaN,-0.843461
10074,SLC22A3,NaN,-0.783325
10075,SSX2B,NaN,NaN


Breast

In [20]:
b = cptac.Brca()

In [21]:
prot = b.get_proteomics()
prot = b.reduce_multiindex(prot, levels_to_drop = 1)
p = list(prot.columns)
len(p)

cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)


10107

In [22]:
mut_type = b.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: Due to dropping the specified levels, dataframe now has 655 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)


Wildtype_Tumor    80
Deletion          24
Name: Mutation, dtype: int64

In [23]:
cols = list(del_wt.columns[:-1])

b_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
b_pval = b_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Brca_P_Value'})
b_pval = b_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#b_pval

In [24]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

b_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    b_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

b_df = pd.DataFrame.from_dict(b_d, orient='index', columns=['Brca_Median'])
b_df = b_df.reset_index().rename(columns={'index':'Proteomics'})
#b_df

In [25]:
b_merged = b_pval.merge(b_df, on='Proteomics',how='inner')
b_merged

,Proteomics,Brca_P_Value,Brca_Median
0,PTEN,0.000004,1.00800
1,EIF4H,0.000025,-0.41295
2,MIEF1,0.000031,-0.84135
3,TMSB10,0.000043,-0.87630
4,DRG1,0.000065,-0.75135
...,...,...,...
9766,POLR3E,0.999356,0.09685
9767,IGFBP7,0.999595,-0.01140
9768,AAMP,0.999613,-0.15430
9769,TRA2B,0.999771,0.17180


Colon

In [26]:
col = cptac.Colon()

In [27]:
prot = col.get_proteomics()
p = list(prot.columns)
len(p)

8067

In [28]:
mut_type = col.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = col.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 100 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 201 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05

Wildtype_Tumor    76
Deletion          25
Name: Mutation, dtype: int64

In [29]:
cols = list(del_wt.columns[:-1])

c_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
c_pval = c_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Colon_P_Value'})
c_pval = c_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#c_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

In [30]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

c_df = pd.DataFrame.from_dict(d, orient='index', columns=['Colon_Median'])
c_df = c_df.reset_index().rename(columns={'index':'Proteomics'})
#c_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [31]:
c_merged = c_pval.merge(c_df, on='Proteomics',how='inner')
c_merged

,Proteomics,Colon_P_Value,Colon_Median
0,DFFA,0.000037,0.19930
1,WAPL,0.000097,0.19605
2,SEC14L2,0.000136,0.66800
3,GBF1,0.000164,0.12630
4,STK11IP,0.000239,0.31250
...,...,...,...
8062,ZNF841,NaN,-0.78250
8063,ZNF888,NaN,NaN
8064,ZNHIT6,NaN,NaN
8065,ZNRD1,NaN,0.06800


Head and Neck

In [32]:
h = cptac.Hnscc()

In [33]:
prot = h.get_proteomics()
p = list(prot.columns)
len(p)

11744

In [34]:
mut_type = h.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
#prot_and_mutations = h.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 109 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-02617, C3N-02727 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 176 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)


Wildtype_Tumor    87
Deletion          18
Name: Mutation, dtype: int64

In [35]:
cols = list(del_wt.columns[:-1])

h_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
h_pval = h_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Hnscc_P_Value'})
h_pval = h_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#h_pval


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

In [36]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

h_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    h_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

h_df = pd.DataFrame.from_dict(h_d, orient='index', columns=['Hnscc_Median'])
h_df = h_df.reset_index().rename(columns={'index':'Proteomics'})
#h_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [37]:
h_merged = h_pval.merge(h_df, on='Proteomics',how='inner')
#del_wt[['PREX2_proteomics','Mutation']].dropna() # in median, not in pval (only 1 value for deletion)
h_merged

,Proteomics,Hnscc_P_Value,Hnscc_Median
0,NLN,1.113430e-09,-0.431666
1,AKR1C3,1.032977e-08,-0.951905
2,AKR1C1,6.931627e-08,-1.961262
3,ADI1,1.601880e-07,-0.513198
4,EPHX1,3.899394e-07,-0.871688
...,...,...,...
11739,ZNF664,NaN,NaN
11740,ZNF836,NaN,NaN
11741,ZNF862,NaN,NaN
11742,ZSCAN31,NaN,NaN


Lung -dropped isoforms so some genes not tested. FIX

In [38]:
l = cptac.Luad()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


In [39]:
prot = l.get_proteomics(tissue_type='tumor')
prot = l.reduce_multiindex(prot, levels_to_drop = 1)
p = list(prot.columns)
len(p)

cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 2)


10699

In [60]:
mut_type = l.get_genotype_all_vars(gene)
mut_type = mut_type.rename(columns={'PTEN':'cnv'})

merged = prot.join(mut_type) # checked and there is 110 tumor samples for lung
merged = merged.loc[:,~merged.columns.duplicated()] # drop duplicate columns (isoforms)

compare = ['No_Mutation','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
print(del_wt['Mutation'].value_counts())
#del_wt.columns[:-2]

No_Mutation    88
Deletion       17
Name: Mutation, dtype: int64


Index(['A1BG', 'A2M', 'AAAS', 'AACS', 'AADAC', 'AADAT', 'AAED1', 'AAGAB',
       'AAMDC', 'AAMP',
       ...
       'ZSCAN31', 'ZSWIM9', 'ZW10', 'ZWILCH', 'ZWINT', 'ZXDC', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='Name', length=10316)

In [70]:
cols = list(del_wt.columns[:-2])

l_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
l_pval = l_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Luad_P_Value'})

l_pval

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

,Proteomics,Luad_P_Value
0,ATE1,2.851800e-07
1,SHOC2,6.424816e-07
2,EGFR,9.121167e-07
3,LANCL2,1.742749e-06
4,GGCT,2.854181e-06
5,CWF19L1,3.173508e-06
6,PPP3CB,3.708740e-06


In [ ]:
#Isoform code - fix later
#del_wt.replace(to_replace = r'_NP_.*$', value = '', regex = True)

In [63]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "No_Mutation"]
del_med = d.median()
wt_med = wt.median()

l_d = {}

for prot in cols:
    dif = wt_med[prot] - del_med[prot]
    l_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

l_df = pd.DataFrame.from_dict(l_d, orient='index', columns=['Luad_Median'])
l_df = l_df.reset_index().rename(columns={'index':'Proteomics'})
#l_df

In [64]:
l_merged = l_pval.merge(l_df, on='Proteomics',how='inner')
l_merged

,Proteomics,Luad_P_Value,Luad_Median
0,ATE1,2.851800e-07,0.88365
1,SHOC2,6.424816e-07,0.60600
2,EGFR,9.121167e-07,-2.32970
3,LANCL2,1.742749e-06,-1.21475
4,GGCT,2.854181e-06,-2.12715
...,...,...,...
10311,COG6,9.997105e-01,0.13015
10312,PLD1,9.997256e-01,-0.43875
10313,CHMP6,9.997659e-01,0.05220
10314,FHL3,9.999702e-01,0.06610


In [65]:
n = g_merged.merge(h_merged, on='Proteomics',how='outer')
n = n.merge(l_merged, on='Proteomics',how='outer')
n= n.merge(b_merged, on='Proteomics',how='outer')
n = n.merge(o_merged, on='Proteomics',how='outer')
n = n.merge(e_merged, on='Proteomics',how='outer')
all_df = n.merge(c_merged, on='Proteomics',how='outer')



In [66]:
all_df

,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,ARMH3,5.396032e-11,0.405134,0.000558,0.135941,0.017908,0.37845,NaN,NaN,NaN,NaN,0.001113,-0.1503,NaN,NaN
1,CUTC,1.593480e-10,0.553255,0.000189,0.314307,0.000744,1.01995,0.008295,0.48080,0.060164,0.159424,0.082868,0.2533,0.411506,0.08210
2,PIP4K2A,1.009419e-09,0.838882,0.001544,0.224713,0.560996,-0.04535,0.203952,0.07435,0.389187,-0.052955,0.201771,-0.1690,0.199847,-0.10135
3,CUL2,1.122076e-09,0.586396,0.400554,-0.017886,0.612583,-0.05695,0.198043,-0.17195,0.786184,-0.033091,0.864381,-0.0232,0.349065,0.07690
4,GDI2,1.302273e-09,0.610188,0.630985,-0.001559,NaN,NaN,0.354712,-0.14995,0.569417,0.067389,0.061660,-0.1286,0.990512,0.04450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14360,WTH3DI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.40100
14361,ZIC4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00200
14362,ZNF419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.38150
14363,ZNF705E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12900


In [90]:
cols = ['Gbm_P_Value','Hnscc_P_Value', 'Luad_P_Value', 'Brca_P_Value','Ov_P_Value','En_P_Value','Colon_P_Value']

bc = .05/8000

t = []

for c in cols:
    df = all_df[all_df[c] <= bc]
    t.append(list(df.Proteomics))
    
flat_list = [item for sublist in t for item in sublist]
sig = set(flat_list) # remove duplicates



bool_df = all_df['Proteomics'].isin(sig)
sig_df = all_df[bool_df]
sig_df


,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,ARMH3,5.396032e-11,0.405134,5.584141e-04,0.135941,0.017908,0.37845,NaN,NaN,NaN,NaN,0.001113,-0.15030,NaN,NaN
1,CUTC,1.593480e-10,0.553255,1.894453e-04,0.314307,0.000744,1.01995,0.008295,0.48080,0.060164,0.159424,0.082868,0.25330,0.411506,0.08210
2,PIP4K2A,1.009419e-09,0.838882,1.543642e-03,0.224713,0.560996,-0.04535,0.203952,0.07435,0.389187,-0.052955,0.201771,-0.16900,0.199847,-0.10135
3,CUL2,1.122076e-09,0.586396,4.005541e-01,-0.017886,0.612583,-0.05695,0.198043,-0.17195,0.786184,-0.033091,0.864381,-0.02320,0.349065,0.07690
4,GDI2,1.302273e-09,0.610188,6.309846e-01,-0.001559,NaN,NaN,0.354712,-0.14995,0.569417,0.067389,0.061660,-0.12860,0.990512,0.04450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10465,SENP2,8.945796e-01,-0.013401,6.561188e-01,-0.086021,0.023553,-0.80130,0.375510,0.38550,0.869603,0.036145,0.000002,0.48150,NaN,NaN
10709,STAT1,9.404646e-01,0.180847,1.305089e-01,0.353813,0.061263,0.60540,0.434161,-0.24190,0.097508,-0.142470,0.000005,0.93700,0.811050,0.06350
11013,PIR,9.948134e-01,0.087051,2.395809e-06,-1.051236,0.176542,-0.20135,0.056007,-1.09030,0.068513,-0.109151,0.184500,-0.19000,0.292827,-0.07240
11141,ADH7,NaN,NaN,4.321698e-07,-2.155736,0.401404,0.12470,NaN,NaN,NaN,NaN,0.079428,-0.52485,NaN,NaN


In [91]:
sig_df.loc[sig_df['Proteomics'] == 'MMS19']

,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
66,MMS19,0.000004,0.324287,0.020464,0.123084,0.000088,0.62,0.019642,0.6534,1.058914e-08,0.324897,0.215233,0.0847,0.005686,0.1218


Combine

In [69]:
s = list(sig)
if 'MMS19' in s:
    print('t')

t


In [68]:
sig_df.to_csv('at_least_one_significant_all.csv')

In [ ]:
stdev = ip_df.std()

In [ ]:
stdev = stdev.to_frame()

In [ ]:
stdev.loc[stdev[0] > .2]

In [ ]:
d_stdev = stdev.std()
d_stdev

In [ ]:
a = stdev.hist(bins = 20)
plt.title("Standard Deviations of Pancancer CNV deletions and amplifications of PTEN on Interacting Proteins\n")
plt.xlabel('StDeV')
plt.ylabel('Count')

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
sns.set(font_scale = 1.3)
a = sns.distplot(stdev, bins = 20)

a.set_title("Standard Deviation Distribution for Pancancer PTEN Mutation Effect on Proteomics\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
a.set(xlabel = 'StDeV', ylabel = 'Frequency')

Mean

In [ ]:
mean_df = stdev.mean()

In [ ]:
a = mean_df.hist(bins = 20)
plt.title("Distribution of the Mean between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Mean')
plt.ylabel('Count')

In [ ]:
m_df = stdev.median()

In [ ]:
a = m_df.hist(bins = 20)
plt.title("Distribution of the Medians between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Median')
plt.ylabel('Count')